In [1]:
from bson.son import SON

In [2]:
from pymongo import MongoClient
import re
import sys

import pandas as pd
import numpy as np

In [3]:
sys.path.insert(0, '../scripts')

In [4]:
import uniformer

In [5]:
client = MongoClient('localhost', 27017)

# BEFORE SUBMITING CHANGE TWITCH_60 IN TWITCH!

In [6]:
db = client.twitch_60

In [7]:
games_coll = db.games
games_coll.find_one()

{'_id': ObjectId('5d05e75c9b63572acde6e5b2'),
 'timestamp': '2019-05-27 18:46:09.148466',
 'data': [{'channels': 14657,
   'game__id': 33214,
   'game_box_large': 'https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-272x380.jpg',
   'game_giantbomb_id': 37030,
   'game_localized_name': 'Fortnite',
   'game_logo_large': 'https://static-cdn.jtvnw.net/ttv-logoart/Fortnite-240x144.jpg',
   'game_name': 'Fortnite',
   'game_norm_name': 'fortnite',
   'game_popularity': 171268,
   'game_unary_name': 'fortnite',
   'viewers': 181860},
  {'channels': 1571,
   'game__id': 32982,
   'game_box_large': 'https://static-cdn.jtvnw.net/ttv-boxart/Grand%20Theft%20Auto%20V-272x380.jpg',
   'game_giantbomb_id': 36765,
   'game_localized_name': 'Grand Theft Auto V',
   'game_logo_large': 'https://static-cdn.jtvnw.net/ttv-logoart/Grand%20Theft%20Auto%20V-240x144.jpg',
   'game_name': 'Grand Theft Auto V',
   'game_norm_name': 'grand theft auto v',
   'game_popularity': 121282,
   'game_unary_name': 'grand the

## Top N games on twitch (collected between 28/05 and 30/05)

Just to check: how many times data from twitch has been collected between 28 and 30 May. We gather all the timestamps

In [8]:
pipeline = [{ "$match" : {'$and' : [
                {'timestamp' : {'$gt' : '2019-05-28'}},
                {'timestamp' : {'$lt' : '2019-05-30'}}
            ]}}, {"$project":
             {'_id': 0,
              'time':'$timestamp'} }]

In [9]:
tsmps = list(games_coll.aggregate(pipeline))

In [10]:
len(tsmps)/ 48

20.0

Data has been collected every 3 minutes, so 20 times each hour is correct.

In [11]:
pipeline = [{"$unwind": "$data"},
            { "$match" : {'$and' : [
                {'timestamp' : {'$gt' : '2019-05-28'}},
                {'timestamp' : {'$lt' : '2019-05-30'}}
            ]}},
            { "$group": {"_id": {"name": "$data.game_name",
                                 "norm_name" : "$data.game_norm_name",
                                 "unary_name" : "$data.game_unary_name",
                                 "twitch_id" : "$data.game__id"},
                         "avgViewers": { "$avg": '$data.viewers'}}},
           {"$sort": SON([("avgViewers", -1), ("_id", -1)])}]

In [12]:
top_twitch = list(db.games.aggregate(pipeline))

In [13]:
top_twitch = list(db.games.aggregate(pipeline))
top_twitch = [
    {'name' : elem['_id']['name'],
     'norm_name' : elem['_id']['norm_name'],
     'unary_name' : elem['_id']['unary_name'],
     'twitch_id' : elem['_id']['twitch_id'],
     'avgViewers' : elem['avgViewers']
    } for elem in top_twitch
]

In [14]:
top_twitch[0:3]

[{'name': 'Fortnite',
  'norm_name': 'fortnite',
  'unary_name': 'fortnite',
  'twitch_id': 33214,
  'avgViewers': 126368.478125},
 {'name': 'League of Legends',
  'norm_name': 'league of legends',
  'unary_name': 'league of legends',
  'twitch_id': 21779,
  'avgViewers': 117746.63333333333},
 {'name': 'Grand Theft Auto V',
  'norm_name': 'grand theft auto v',
  'unary_name': 'grand theft auto v',
  'twitch_id': 32982,
  'avgViewers': 94222.24895833334}]

In [15]:
top_twitch_df = pd.DataFrame(top_twitch)

# Top N games on Vgchartz from Mongo DB

It is necessary to execute the Notebook "Save Top N games on MongoDB" before executing the next code chunks

In [16]:
db_vg = client.vgchartz
top_games_vg = db_vg.top_games

Quick check for seeing if everything was loaded correctly

In [17]:
top_games_vg.find_one()

{'_id': ObjectId('5d00d5385024ccaf12d8137e'),
 'Rank': 1,
 'Name': 'Wii Sports',
 'Platform': 'Wii',
 'Year': 2006.0,
 'href_Genre': 'http://www.vgchartz.com/game/2667/wii-sports/?region=All',
 'Publisher': 'Nintendo  ',
 'Developer': 'Nintendo EAD  ',
 'Critic_Score': 7.7,
 'User_Score': nan,
 'NA_Sales': 41.36,
 'PAL_Sales': 29.02,
 'JP_Sales': 3.77,
 'Other_Sales': 8.51,
 'Global_Sales': 82.65}

Vgchartz returns the global sales data for each game and for each Platform. Therefore the game "Grand Theft Auto V" will appear several times. Let's find out if this is correct.

In [18]:
gtaV_occurences = top_games_vg.find({"Name": "Grand Theft Auto V"})
for game in gtaV_occurences:
    print(game)

{'_id': ObjectId('5d00d5385024ccaf12d8138f'), 'Rank': 20, 'Name': 'Grand Theft Auto V', 'Platform': 'PS3', 'Year': 2013.0, 'href_Genre': 'http://www.vgchartz.com/game/65884/grand-theft-auto-v/?region=All', 'Publisher': 'Rockstar Games  ', 'Developer': 'Rockstar North  ', 'Critic_Score': 9.4, 'User_Score': nan, 'NA_Sales': 6.37, 'PAL_Sales': 9.85, 'JP_Sales': 0.99, 'Other_Sales': 3.12, 'Global_Sales': 20.32}
{'_id': ObjectId('5d00d5385024ccaf12d81390'), 'Rank': 21, 'Name': 'Grand Theft Auto V', 'Platform': 'PS4', 'Year': 2014.0, 'href_Genre': 'http://www.vgchartz.com/game/83196/grand-theft-auto-v/?region=All', 'Publisher': 'Rockstar Games  ', 'Developer': 'Rockstar North  ', 'Critic_Score': 9.7, 'User_Score': nan, 'NA_Sales': 6.06, 'PAL_Sales': 9.71, 'JP_Sales': 0.6, 'Other_Sales': 3.02, 'Global_Sales': 19.39}
{'_id': ObjectId('5d00d5385024ccaf12d8139c'), 'Rank': 34, 'Name': 'Grand Theft Auto V', 'Platform': 'X360', 'Year': 2013.0, 'href_Genre': 'http://www.vgchartz.com/game/65883/grand

As imagined there is one data entry for each platform. We are interested in the "whole" game performance, therefore we will perform the following actions:
- Group by the games according to their name.
- Sum all the global sales
- Average the Critic Score
- Average the User Score
- Store all Platforms: as in the GTA V case many games were released for different platforms, therefore it is important to not lose this info after the group by
- Store all the game developers: it is possible that the games that were released for different platforms were developed by different software houses, therefore it is importat to not lose this info after the group by


The following pipeline perform as first operation the creation of two attributes (Critic_Score_norm and User_Score_norm) where all the NaN were replaced with null. In this way they aren't considered the average operation doesn't consider them.

In [19]:
vgchartz_pipeline = [
     { "$addFields": {
         "Critic_Score_norm": {"$cond": 
             { "if": { "$eq": [ "$Critic_Score", np.nan ] }, 
                   "then": "null", 
                   "else": "$Critic_Score" }},
         "User_Score_norm": {"$cond": 
             { "if": { "$eq": [ "$User_Score", np.nan ] }, 
                   "then": "null", 
                   "else": "$User_Score" }}
        }
     }, 
    { "$group": {
        "_id": "$Name",
        "Name": {"$first":"$Name"}, #This is necessary because every variable assignment should be done using group operators
        "Global_sales_all_platform": {"$sum": "$Global_Sales"},
        "Year": {"$first": "$Year"},
        "Critic_score": {"$avg":"$Critic_Score_norm"},
        "User_score": {"$avg":"$User_Score_norm"},
        "Publishers": {"$push": {"$concat":["$Publisher"]}},
        "Developers": {"$push": {"$concat":["$Developer"]}},
        "Platforms": {"$push": {"$concat":["$Platform"]}}
        }
    },
    { "$project": { "_id": 0 } },
    {"$sort": {"Global_sales_all_platform": -1} }]
top_vgchartz = list(top_games_vg.aggregate(vgchartz_pipeline))

In [20]:
vgchartz_df = pd.DataFrame.from_dict(top_vgchartz)
vgchartz_df.head(5)

,Critic_score,Developers,Global_sales_all_platform,Name,Platforms,Publishers,User_score,Year
0,7.700000,[Nintendo EAD ],82.65,Wii Sports,[Wii],[Nintendo ],NaN,2006.0
1,9.366667,"[Rockstar North , Rockstar North , Rockstar ...",65.62,Grand Theft Auto V,"[PS3, PS4, X360, PC, XOne]","[Rockstar Games , Rockstar Games , Rockstar ...",9.0,2013.0
2,10.000000,[Nintendo EAD ],40.24,Super Mario Bros.,[NES],[Nintendo ],8.2,1985.0
3,8.200000,[Nintendo EAD ],35.98,Mario Kart Wii,[Wii],[Nintendo ],9.1,2008.0
4,NaN,"[Bullet Proof Software , Nintendo ]",35.84,Tetris,"[GB, NES]","[Nintendo , Nintendo ]",NaN,1989.0


# Join Vgchartz and Twitch Data

Let's join the data and see the first 25 games

In [21]:
merged_df = pd.merge(vgchartz_df, top_twitch_df,  left_on = 'Name', right_on = 'norm_name', how = 'outer')
merged_df[['Name', 'twitch_id', 'norm_name', 'Name']].loc[0:24]

,Name,twitch_id,norm_name,Name
0,Wii Sports,NaN,NaN,Wii Sports
1,Grand Theft Auto V,NaN,NaN,Grand Theft Auto V
2,Super Mario Bros.,NaN,NaN,Super Mario Bros.
3,Mario Kart Wii,NaN,NaN,Mario Kart Wii
4,Tetris,NaN,NaN,Tetris
5,Minecraft,NaN,NaN,Minecraft
6,Wii Sports Resort,NaN,NaN,Wii Sports Resort
7,Pokémon Red / Green / Blue Version,NaN,NaN,Pokémon Red / Green / Blue Version
8,Call of Duty: Black Ops,NaN,NaN,Call of Duty: Black Ops
9,Call of Duty: Modern Warfare 3,NaN,NaN,Call of Duty: Modern Warfare 3


As you can see the games may be there is no match. The reason why is that on twitch the games are all in lower case, without punctuation and with arabic numbers converted into roman letters.

In [22]:
top_twitch_df.head(5)

,avgViewers,name,norm_name,twitch_id,unary_name
0,126368.478125,Fortnite,fortnite,33214,fortnite
1,117746.633333,League of Legends,league of legends,21779,league of legends
2,94222.248958,Grand Theft Auto V,grand theft auto v,32982,grand theft auto v
3,79284.045833,Just Chatting,just chatting,509658,just chatting
4,68005.006250,Dota 2,dota ii,29595,dota ii


For this reason we will use a specific custom function that will "normalize" the names.

In [23]:
vgchartz_df['Norm_name_vgchartz'] = vgchartz_df['Name'].apply(lambda x: uniformer.uniform(x))

In [24]:
merged_df = pd.merge(vgchartz_df, top_twitch_df,  left_on = 'Norm_name_vgchartz', right_on = 'norm_name', how = 'outer').loc[0:50]
merged_df[['Name', 'twitch_id', 'norm_name', 'Norm_name_vgchartz']]

,Name,twitch_id,norm_name,Norm_name_vgchartz
0,Wii Sports,12232.0,wii sports,wii sports
1,Grand Theft Auto V,32982.0,grand theft auto v,grand theft auto v
2,Super Mario Bros.,509508.0,super mario bros.,super mario bros.
3,Mario Kart Wii,18871.0,mario kart wii,mario kart wii
4,Tetris,1736.0,tetris,tetris
5,Minecraft,27471.0,minecraft,minecraft
6,Wii Sports Resort,19259.0,wii sports resort,wii sports resort
7,Pokémon Red / Green / Blue Version,NaN,NaN,pokémon red / green / blue version
8,Call of Duty: Black Ops,23894.0,call of duty black ops,call of duty black ops
9,Call of Duty: Modern Warfare 3,31551.0,call of duty modern warfare iii,call of duty modern warfare iii


Let's check which matches are missing:

In [25]:
merged_df[merged_df['norm_name'].isna()][['Norm_name_vgchartz']]

,Norm_name_vgchartz
7,pokémon red / green / blue version
17,call of duty modern warfare ii
18,nintendogs
21,pokémon gold / silver version
22,wii fit
24,kinect adventures!
25,wii fit plus
29,brain age train your brain in minutes a day
33,pokémon diamond / pearl version
45,pokémon ruby / sapphire version


- pokemon red / green / blue is called different

In [26]:
top_twitch_df[top_twitch_df['norm_name'].str.contains('pokémon red')]

,avgViewers,name,norm_name,twitch_id,unary_name
1096,31.33956,Pokémon Red/Blue,pokémon red/blue,3623,pokémon red/blue


Because I can't see them I will try a broader query on twitch collection and try to retrieve the names

In [27]:
vgchartz_df.loc[vgchartz_df.Norm_name_vgchartz == "pokémon red / green / blue version", "Norm_name_vgchartz"] = 'pokémon red/blue'

- call of duty modern warfare ii is called slightly different

In [28]:
top_twitch_df[top_twitch_df['norm_name'].str.contains('modern warfare')]

,avgViewers,name,norm_name,twitch_id,unary_name
436,165.033684,Modern Warfare 2,modern warfare ii,22393,modern warfare ii
556,109.727561,Call of Duty: Modern Warfare Remastered,call of duty modern warfare remastered,494155,call of duty modern warfare remastered
1128,29.654709,Call of Duty: Modern Warfare 3,call of duty modern warfare iii,31551,call of duty modern warfare iii
1196,27.126836,Call of Duty 4: Modern Warfare,call of duty 4 modern warfare,1964,call of duty 4 modern warfare
1365,21.397849,Call of Duty: Modern Warfare,call of duty modern warfare,512710,call of duty modern warfare
7550,0.507289,Call of Duty: Modern Warfare: Mobilized,call of duty modern warfare mobilized,24951,call of duty modern warfare mobilized


In [29]:
vgchartz_df.loc[vgchartz_df.Norm_name_vgchartz == "call of duty modern warfare ii", "Norm_name_vgchartz"] = 'modern warfare ii'

- nintendogs is not on twitch

In [30]:
top_twitch_df[top_twitch_df['norm_name'].str.contains('nintend')]

,avgViewers,name,norm_name,twitch_id,unary_name
54,3212.307692,Nintendo Land,nintendo land,65960,nintendo land
107,1207.275362,Monopoly for Nintendo Switch,monopoly for nintendo switch,496604,monopoly for nintendo switch
1713,14.360825,Super Smash Bros. for Nintendo 3DS,super smash bros. for nintendo 3ds,489023,super smash bros. for nintendo 3ds
2527,7.700000,Pokémon for Nintendo Switch (Working Title),pokémon for nintendo switch (working title),497451,pokémon for nintendo switch (working title)
2734,6.835443,Nintendo Fan Network,nintendo fan network,491512,nintendo fan network
3679,4.273973,Taiko no Tatsujin: Nintendo Switch Version,taiko no tatsujin nintendo switch version,504454,taiko no tatsujin nintendo switch version
4756,2.709091,Nintendo Labo Variety Kit,nintendo labo variety kit,502913,nintendo labo variety kit
4989,2.444444,Dragon Quest Heroes I・II for Nintendo Switch,dragon quest heroes i・ii for nintendo switch,495059,dragon quest heroes i・ii for nintendo switch
5082,2.333333,Nintendo DSi Instrument Tuner,nintendo dsi instrument tuner,25237,nintendo dsi instrument tuner


- wii fit/ wii fit plus are not on twitch

In [31]:
top_twitch_df[top_twitch_df['norm_name'].str.contains('wii')]

,avgViewers,name,norm_name,twitch_id,unary_name
602,93.326776,Mario Kart Wii,mario kart wii,18871,mario kart wii
740,63.221985,Call of Duty: WWII,call of duty wwii,496712,call of duty wwii
806,54.773810,Strategic Command WWII: World at War,strategic command wwii world at war,510548,strategic command wwii world at war
1445,19.032258,PokéPark Wii: Pikachu's Adventure,poképark wii pikachu's adventure,28024,poképark wii pikachu's adventure
1969,11.296296,Wii Sports Club,wii sports club,408033,wii sports club
2043,10.728448,New Super Mario Bros. Wii,new super mario bros. wii,24238,new super mario bros. wii
3391,4.881188,Wii Sports,wii sports,12232,wii sports
4243,3.333333,Wii Party U,wii party u,110751,wii party u
4434,3.050657,Super Smash Bros. for Wii U,super smash bros. for wii u,488353,super smash bros. for wii u
6149,1.500000,Mario Kart Wii U (Working Title),mario kart wii u (working title),110753,mario kart wii u (working title)


- knect adventures is not on twitch

In [32]:
top_twitch_df[top_twitch_df['norm_name'].str.contains('kinect')]

,avgViewers,name,norm_name,twitch_id,unary_name
3569,4.483871,Kinect Disneyland Adventures,kinect disneyland adventures,31884,kinect disneyland adventures
5502,2.000000,Kinect Rush: A Disney/Pixar Adventure,kinect rush a disney/pixar adventure,32939,kinect rush a disney/pixar adventure
5942,1.666667,Harry Potter for Kinect,harry potter for kinect,65831,harry potter for kinect
6972,1.000000,Fruit Ninja Kinect 2,fruit ninja kinect ii,461397,fruit ninja kinect ii


- brain age/ brain age 2 are not on twitch

In [33]:
top_twitch_df[top_twitch_df['norm_name'].str.contains('brain')]

,avgViewers,name,norm_name,twitch_id,unary_name
1544,17.117647,Brain / Out,brain / out,496037,brain / out
2466,8.000000,Brain Lord,brain lord,1447,brain lord
5343,2.097087,The Island of Dr. Brain,the island of dr. brain,16134,the island of dr. brain


Solving the remaining Pokemon discrepancies:

In [34]:
top_twitch_df[top_twitch_df['norm_name'].str.contains('ruby')]

,avgViewers,name,norm_name,twitch_id,unary_name
582,100.829940,Pokémon Ruby/Sapphire,pokémon ruby/sapphire,9287,pokémon ruby/sapphire
1600,16.106299,Pokémon Omega Ruby/Alpha Sapphire,pokémon omega ruby/alpha sapphire,460327,pokémon omega ruby/alpha sapphire
5969,1.636364,Pokémon Pinball: Ruby & Sapphire,pokémon pinball ruby & sapphire,8075,pokémon pinball ruby & sapphire


In [35]:
vgchartz_df.loc[vgchartz_df.Norm_name_vgchartz == "pokémon ruby / sapphire version", 'Norm_name_vgchartz'] = 'pokémon ruby/sapphire'

In [36]:
top_twitch_df[top_twitch_df['norm_name'].str.contains('pearl')]

,avgViewers,name,norm_name,twitch_id,unary_name
1549,17.008197,Pokémon Diamond/Pearl,pokémon diamond/pearl,2993,pokémon diamond/pearl


In [37]:
vgchartz_df.loc[vgchartz_df.Norm_name_vgchartz == "pokémon diamond / pearl version", "Norm_name_vgchartz"] = 'pokémon diamond/pearl'

In [38]:
top_twitch_df[top_twitch_df['norm_name'].str.contains('pokémon gold')]

,avgViewers,name,norm_name,twitch_id,unary_name
1114,30.186462,Pokémon Gold/Silver,pokémon gold/silver,7528,pokémon gold/silver


In [39]:
vgchartz_df.loc[vgchartz_df.Norm_name_vgchartz == "pokémon gold / silver version", "Norm_name_vgchartz"] = 'pokémon gold/silver'

- Let's see if Fifa 13 is available on twitch

In [40]:
top_twitch_df[top_twitch_df['norm_name'].str.contains('fifa')]

,avgViewers,name,norm_name,twitch_id,unary_name
20,10657.876042,FIFA 19,fifa xix,506103,fifa iiiiiiiiiiiiiiiiiii
245,416.171717,FIFA Soccer 97,fifa soccer xcvii,3107,fifa soccer iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii...
368,217.278912,FIFA 11,fifa xi,28500,fifa iiiiiiiiiii
379,203.572939,FIFA 17,fifa xvii,493091,fifa iiiiiiiiiiiiiiiii
601,93.448649,FIFA 15,fifa xv,460402,fifa iiiiiiiiiiiiiii
641,82.997126,FIFA Online 4,fifa online iv,504798,fifa online iiii
1173,28.109560,FIFA 18,fifa xviii,495589,fifa iiiiiiiiiiiiiiiiii
4785,2.666667,FIFA Soccer 13,fifa soccer xiii,32443,fifa soccer iiiiiiiiiiiii
5521,2.000000,FIFA 2000,fifa mm,26486,fifa iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii...
5747,1.835821,FIFA 16,fifa xvi,489608,fifa iiiiiiiiiiiiiiii


In [41]:
vgchartz_df.loc[vgchartz_df.Norm_name_vgchartz == "fifa xiii", "Norm_name_vgchartz"] = 'fifa soccer xiii'

Now the main discrepancies are solved, let's rejoin the datasets

## Merge data

In [42]:
top_twitch_vgchartz = pd.merge(vgchartz_df, top_twitch_df,  left_on = 'Norm_name_vgchartz', right_on = 'norm_name', how = 'outer').loc[0:50]

In [43]:
top_twitch_vgchartz.head(25)

,Critic_score,Developers,Global_sales_all_platform,Name,Platforms,Publishers,User_score,Year,Norm_name_vgchartz,avgViewers,name,norm_name,twitch_id,unary_name
0,7.700000,[Nintendo EAD ],82.65,Wii Sports,[Wii],[Nintendo ],NaN,2006.0,wii sports,4.881188,Wii Sports,wii sports,12232.0,wii sports
1,9.366667,"[Rockstar North , Rockstar North , Rockstar ...",65.62,Grand Theft Auto V,"[PS3, PS4, X360, PC, XOne]","[Rockstar Games , Rockstar Games , Rockstar ...",9.0,2013.0,grand theft auto v,94222.248958,Grand Theft Auto V,grand theft auto v,32982.0,grand theft auto v
2,10.000000,[Nintendo EAD ],40.24,Super Mario Bros.,[NES],[Nintendo ],8.2,1985.0,super mario bros.,49.296454,Super Mario Bros.,super mario bros.,509508.0,super mario bros.
3,8.200000,[Nintendo EAD ],35.98,Mario Kart Wii,[Wii],[Nintendo ],9.1,2008.0,mario kart wii,93.326776,Mario Kart Wii,mario kart wii,18871.0,mario kart wii
4,NaN,"[Bullet Proof Software , Nintendo ]",35.84,Tetris,"[GB, NES]","[Nintendo , Nintendo ]",NaN,1989.0,tetris,50.957424,Tetris,tetris,1736.0,tetris
5,7.700000,"[Mojang , Mojang , Mojang , Mojang , 4J St...",33.83,Minecraft,"[X360, PS4, PS3, XOne, PSV, NS, WiiU, 3DS]","[Microsoft Studios , Sony Computer Entertainm...",NaN,2013.0,minecraft,12378.073958,Minecraft,minecraft,27471.0,minecraft
6,8.000000,[Nintendo EAD ],32.90,Wii Sports Resort,[Wii],[Nintendo ],8.8,2009.0,wii sports resort,1.351852,Wii Sports Resort,wii sports resort,19259.0,wii sports resort
7,9.400000,[Game Freak ],31.37,Pokémon Red / Green / Blue Version,[GB],[Nintendo ],NaN,1998.0,pokémon red/blue,31.339560,Pokémon Red/Blue,pokémon red/blue,3623.0,pokémon red/blue
8,8.100000,"[Treyarch , Treyarch , Treyarch , Treyarch ...",30.99,Call of Duty: Black Ops,"[X360, PS3, PC, Wii, DS]","[Activision , Activision , Activision , Act...",NaN,2010.0,call of duty black ops,394.394931,Call of Duty: Black Ops,call of duty black ops,23894.0,call of duty black ops
9,7.500000,"[Infinity Ward , Infinity Ward , Infinity Wa...",30.71,Call of Duty: Modern Warfare 3,"[X360, PS3, PC, Wii]","[Activision , Activision , Activision , Act...",NaN,2011.0,call of duty modern warfare iii,29.654709,Call of Duty: Modern Warfare 3,call of duty modern warfare iii,31551.0,call of duty modern warfare iii


We have decided to further monitor the following data:
- Top 25 games from vgchartz (excluding the ones that aren't on twitch)
- Top 25 games from twitch (excluding the ones that were previously selected)

### Select top 25 from vgchartz

In [44]:
top_25_vgchartz = top_twitch_vgchartz[top_twitch_vgchartz.norm_name.notnull()].reset_index().loc[0:25]

In [45]:
top_25_vgchartz

,index,Critic_score,Developers,Global_sales_all_platform,Name,Platforms,Publishers,User_score,Year,Norm_name_vgchartz,avgViewers,name,norm_name,twitch_id,unary_name
0,0,7.700000,[Nintendo EAD ],82.65,Wii Sports,[Wii],[Nintendo ],NaN,2006.0,wii sports,4.881188,Wii Sports,wii sports,12232.0,wii sports
1,1,9.366667,"[Rockstar North , Rockstar North , Rockstar ...",65.62,Grand Theft Auto V,"[PS3, PS4, X360, PC, XOne]","[Rockstar Games , Rockstar Games , Rockstar ...",9.0,2013.0,grand theft auto v,94222.248958,Grand Theft Auto V,grand theft auto v,32982.0,grand theft auto v
2,2,10.000000,[Nintendo EAD ],40.24,Super Mario Bros.,[NES],[Nintendo ],8.2,1985.0,super mario bros.,49.296454,Super Mario Bros.,super mario bros.,509508.0,super mario bros.
3,3,8.200000,[Nintendo EAD ],35.98,Mario Kart Wii,[Wii],[Nintendo ],9.1,2008.0,mario kart wii,93.326776,Mario Kart Wii,mario kart wii,18871.0,mario kart wii
4,4,NaN,"[Bullet Proof Software , Nintendo ]",35.84,Tetris,"[GB, NES]","[Nintendo , Nintendo ]",NaN,1989.0,tetris,50.957424,Tetris,tetris,1736.0,tetris
5,5,7.700000,"[Mojang , Mojang , Mojang , Mojang , 4J St...",33.83,Minecraft,"[X360, PS4, PS3, XOne, PSV, NS, WiiU, 3DS]","[Microsoft Studios , Sony Computer Entertainm...",NaN,2013.0,minecraft,12378.073958,Minecraft,minecraft,27471.0,minecraft
6,6,8.000000,[Nintendo EAD ],32.90,Wii Sports Resort,[Wii],[Nintendo ],8.8,2009.0,wii sports resort,1.351852,Wii Sports Resort,wii sports resort,19259.0,wii sports resort
7,7,9.400000,[Game Freak ],31.37,Pokémon Red / Green / Blue Version,[GB],[Nintendo ],NaN,1998.0,pokémon red/blue,31.339560,Pokémon Red/Blue,pokémon red/blue,3623.0,pokémon red/blue
8,8,8.100000,"[Treyarch , Treyarch , Treyarch , Treyarch ...",30.99,Call of Duty: Black Ops,"[X360, PS3, PC, Wii, DS]","[Activision , Activision , Activision , Act...",NaN,2010.0,call of duty black ops,394.394931,Call of Duty: Black Ops,call of duty black ops,23894.0,call of duty black ops
9,9,7.500000,"[Infinity Ward , Infinity Ward , Infinity Wa...",30.71,Call of Duty: Modern Warfare 3,"[X360, PS3, PC, Wii]","[Activision , Activision , Activision , Act...",NaN,2011.0,call of duty modern warfare iii,29.654709,Call of Duty: Modern Warfare 3,call of duty modern warfare iii,31551.0,call of duty modern warfare iii


# Select top 25 games from Twitch